GET: Retrieve data. This is a read-only operation. It should be safe (no side-effects) and idempotent (making the same call multiple times produces the same result).

Example: GET /users/123 (Get information for user with ID 123).

POST: Create a new resource. This is not safe and not idempotent.

Example: POST /users (Create a new user with the data provided in the request body). Each successful POST should create a new, distinct user.

PUT: Replace an existing resource in its entirety. You send the complete, updated representation of the resource. It's idempotent.

Example: PUT /users/123 (Replace user 123's entire profile with new data). If you send the same request twice, the final state of the resource is the same.

PATCH: Partially update an existing resource. You only send the fields that need to change. It is generally not idempotent.

Example: PATCH /users/123 (Update only the email address for user 123).

DELETE: Remove a resource. It is idempotent.

Example: DELETE /users/123 (Delete user 123). Calling it again on a deleted resource still results in a "does not exist" state.

HTTP Status Codes: The Server's Feedback
The server includes a status code in its response to tell the client the outcome of its request. They are grouped into families:

2xx (Success)

200 OK: The request was successful. The standard response for GET, PUT, PATCH.

201 Created: The request was successful, and a resource was created. The standard response for a successful POST.

204 No Content: The request was successful, but there is no data to send back. Common for DELETE.

4xx (Client Error) - "You messed up."

400 Bad Request: The server could not understand the request due to invalid syntax (e.g., malformed JSON).

401 Unauthorized: You are not authenticated. You need to log in to access this.

403 Forbidden: You are authenticated, but you don't have permission to perform this action.

404 Not Found: The resource you requested does not exist.

422 Unprocessable Entity: The request syntax was correct, but the data was semantically flawed (e.g., an email field was provided, but it wasn't a valid email address). FastAPI uses this extensively.

5xx (Server Error) - "I messed up."

500 Internal Server Error: A generic, unexpected error occurred on the server. This is your cue to check the server logs.

</br>
</br>
</br>
</br>
</br>
REST (REpresentational State Transfer) Principles
REST is not a protocol, but an architectural style. An API is "RESTful" if it adheres to these constraints, which lead to scalable and maintainable systems.

Client-Server Architecture: The client (front-end) and server (back-end) are separated. They evolve independently and communicate over the network. This is the foundation of modern application development.

Statelessness: As mentioned before, every request from the client must contain all the information necessary for the server to process it. The server does not store any client session state. This makes the application more reliable, scalable, and easier to cache.

Cacheability: Responses should implicitly or explicitly define themselves as cacheable or not. This allows clients or intermediaries to reuse response data, dramatically improving performance.

Uniform Interface: This is the core principle that simplifies and decouples the architecture. It has a few parts:

Resources are identified by URIs (e.g., /users/123).

You manipulate resources through their representations (e.g., the JSON you GET or PUT).

The HTTP methods (GET, POST, DELETE, etc.), headers, and status codes provide a common, well-understood language for interaction.

## OpenAPI
FastAPI automatically generates a schema that conforms to the OpenAPI Specification. OpenAPI is an open standard for defining REST APIs. This schema is a big JSON object that describes every part of your API:

Every available endpoint (e.g., /items/{item_id}).

The HTTP methods for each endpoint (GET, POST, etc.).

The parameters for each endpoint (path, query, headers).

The structure of request bodies and responses.

## Swagger UI
This is the most common and powerful documentation interface. It turns your API definition into a mini, interactive website.</br>
localhost:8000/docs</br>
localhost:8000/redoc


### Feild
lets you create more constraints on the model variables

## Exceptions
FastAPI provides a special exception class, HTTPException, for this exact purpose. When you raise an HTTPException from within your endpoint's code, FastAPI will halt processing and immediately send a formatted HTTP error response.



## Common Parameters
 A Simple Function Dependency
Imagine you have common query parameters for pagination that you want to use in multiple endpoints. Instead of repeating them in every function signature, you can extract them into a dependency.

## yield
Dependencies with yield: Setup and Teardown
This is the most powerful use case for DI. What if a dependency needs to perform some action before the endpoint runs (like opening a database connection) and some cleanup action after it's finished (like closing the connection)?

A dependency function using yield can achieve this. The code before the yield is the "setup" part, and the code after is the "teardown" part. The teardown code is guaranteed to run, even if an error occurs in the endpoint.

## DataBase
An Object-Relational Mapper (ORM) solves this. It maps the Python objects you love to the relational tables in your database. You can interact with your data using Python classes and methods, and the ORM handles the secure and efficient generation of SQL for you.

SQLAlchemy is the preeminent ORM in the Python ecosystem. It is incredibly powerful, mature, and, most importantly for us, its latest versions have first-class support for asyncio, making it a perfect match for FastAPI.

## Why Migrations? Why Alembic?
Your application will evolve. You'll need to add a new column to a table, create a new table, or change a data type. How do you apply these changes to your production database schema without losing data?

This process is called database migration. Alembic is the migration tool for SQLAlchemy. It provides a version control system for your database schema, just like Git does for your code.

The workflow is:

You change your SQLAlchemy models in your Python code.

You run an Alembic command to autogenerate a migration script.

You review the script, then run another command to apply it to your database, safely altering its structure.

## Asynchronous CRUD Operations

## Authorization
The modern, stateless solution is the JSON Web Token (JWT).

After a successful login, the server generates a JWT and gives it to the client. The client then includes this token (usually in an Authorization header) with every subsequent request to prove its identity.

A JWT looks like a long string of random characters, separated by two dots: xxxxx.yyyyy.zzzzz

This string has three parts:

1. Header (the xxxxx part)

A Base64Url-encoded JSON object describing the token.

Contains the hashing algorithm used ("alg": "HS256") and the token type ("typ": "JWT").

2. Payload (the yyyyy part)

A Base64Url-encoded JSON object containing the "claims" – the actual data.

This is where you put the user's identity, like their user ID (sub), when the token expires (exp), and any other data you need, like their username or roles.

Example: {"sub": "123", "username": "alex", "exp": 1660000000}.

3. Signature (the zzzzz part)

This is the security mechanism. It is created by taking the encoded Header, the encoded Payload, a secret key (known only to your server), and signing them with the algorithm from the header.

</br>
</br>
Our goal is to create a system where a user can "log in" to receive a token, and then use that token to access protected resources. We will follow the OAuth2 password flow, which is standard for first-party clients (like a mobile app or a single-page web app talking to its own backend).

## WebSockets
The WebSocket Endpoint
In FastAPI, you create a WebSocket endpoint using the @app.websocket() decorator. Instead of receiving standard request and response objects, your function will receive a WebSocket object that you use to communicate.

Let's break down the websocket_endpoint function:

await websocket.accept(): This is mandatory. It performs the WebSocket "handshake" and establishes the persistent connection.

while True:: The connection is persistent, so we enter a loop to continuously listen for communication.

await websocket.receive_text(): This line waits until a text message is received from the client. FastAPI also provides receive_json() and receive_bytes().

await websocket.send_text(...): This sends a text message back to the connected client.

except WebSocketDisconnect:: If the client closes the connection (e.g., closes the browser tab), receive_text() will raise a WebSocketDisconnect exception. We catch this to exit our loop gracefully instead of having the server crash.